<a href="https://colab.research.google.com/github/benevolent-machines/datamachine-example/blob/main/module_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#test
!pip install datamachine

In [ ]:
#test
import datamachine as dm
import plotly.io as pio
pio.renderers.default='colab'  # this is modified in background generation

In [ ]:
from IPython.display import display # avoid problems in machine

In [ ]:
class Data:
    # https://www.irs.gov/charities-non-profits/exempt-organizations-business-master-file-extract-eo-bmf
    _eomf_dt = {  # Exempt Organization Master file
        "EIN": "string",
        "NAME": "string",
        "ICO": "string",
        "STREET": "string",
        "CITY": "string",
        "STATE": "string",
        "ZIP": "string",
        "RULING": "string",
        "TAX_PERIOD": "string",
        "GROUP": "string",
        "SUBSECTION": "string",
        "AFFILIATION": "string",
        "CLASSIFICATION": "string",
        "RULING": "string",
        "DEDUCTIBILITY": "string",
        "FOUNDATION": "string",
        "ACTIVITY": "string",
        "ORGANIZATION": "string",
        "STATUS": "string",
        "TAX_PERIOD": "string",
        "ASSET_CD": "string",
        "INCOME_CD": "string",
        "FILING_REQ_CD": "string",
        "PF_FILING_REQ_CD": "string",
        "ACCT_PD": "string",
        "ASSET_AMT": "Int64",
        "INCOME_AMT": "Int64",
        "REVENUE_AMT": "Int64",
        "NTEE_CD": "string",
        "SORT_NAME": "string",
    }
    # extract of https://www.irs.gov/charities-non-profits/form-990-series-downloads
    _e990_dt = {
        'BLOBI': 'string',
        'EIN': 'string',
        'ReturnTypeCd': 'string',
        'TaxPeriodEndDt': 'string',
        'BusinessName': 'string',
        'BusinessStreet': 'string',
        'CityNm': 'string',
        'StateAbbreviationCd': 'string',
        'ZIPCd': 'string',
        'WebsiteAddressTxt': 'string',
        'TotalEmployeeCnt': 'Int32',
        'TotalVolunteersCnt': 'Int32',
        'GrossReceiptsAmt': 'Int64',
        'PYContributionsGrantsAmt': 'Int64',
        'CYContributionsGrantsAmt': 'Int64',
        'PYProgramServiceRevenueAmt': 'Int64',
        'CYProgramServiceRevenueAmt': 'Int64',
        'PYInvestmentIncomeAmt': 'Int64',
        'CYInvestmentIncomeAmt': 'Int64',
        'PYOtherRevenueAmt': 'Int64',
        'CYOtherRevenueAmt': 'Int64',
        'PYTotalRevenueAmt': 'Int64',
        'CYTotalRevenueAmt': 'Int64',
        'PYGrantsAndSimilarPaidAmt': 'Int64',
        'CYGrantsAndSimilarPaidAmt': 'Int64',
        'PYBenefitsPaidToMembersAmt': 'Int64',
        'CYBenefitsPaidToMembersAmt': 'Int64',
        'PYSalariesCompEmpBnftPaidAmt': 'Int64',
        'CYSalariesCompEmpBnftPaidAmt': 'Int64',
        'PYTotalProfFndrsngExpnsAmt': 'Int64',
        'CYTotalProfFndrsngExpnsAmt': 'Int64',
        'CYTotalFundraisingExpenseAmt': 'Int64',
        'PYOtherExpensesAmt': 'Int64',
        'CYOtherExpensesAmt': 'Int64',
        'PYTotalExpensesAmt': 'Int64',
        'CYTotalExpensesAmt': 'Int64',
        'PYRevenuesLessExpensesAmt': 'Int64',
        'CYRevenuesLessExpensesAmt': 'Int64',
        'TotalAssetsBOYAmt': 'Int64',
        'TotalAssetsEOYAmt': 'Int64',
        'TotalLiabilitiesBOYAmt': 'Int64',
        'TotalLiabilitiesEOYAmt': 'Int64',
        'NetAssetsOrFundBalancesBOYAmt': 'Int64',
        'NetAssetsOrFundBalancesEOYAmt': 'Int64',
        'ActivityOrMissionDesc': 'string'
    }
    # https://raw.githubusercontent.com/dhenderson/ntee/master/ntee.json
    _ntee_dt = {
        'code': 'string',
        'description': 'string',
        'keywords': 'string',
        'title': 'string'
    }
    #
    _eomf = None # exempt organizations master file
    _e990 = None # electronic 990 records
    _ntee = None # national taxonomy of exempt entities
    _gcst = None # candid grant classification system - present in 990 NTEE codes
    _activities = None # fundraising activities from schedule G
    def get_eomf():
        import pandas as pd
        if Data._eomf is None:
            eo1 = pd.read_csv('https://www.irs.gov/pub/irs-soi/eo1.csv', dtype=Data._eomf_dt)
            eo2 = pd.read_csv('https://www.irs.gov/pub/irs-soi/eo2.csv', dtype=Data._eomf_dt)
            eo3 = pd.read_csv('https://www.irs.gov/pub/irs-soi/eo3.csv', dtype=Data._eomf_dt)
            eo4 = pd.read_csv('https://www.irs.gov/pub/irs-soi/eo4.csv', dtype=Data._eomf_dt)
            Data._eomf = pd.concat([eo1,eo2,eo3,eo4])
        return Data._eomf
    def get_e990():
        import pandas as pd
        if Data._e990 is None:
            u = 'https://storage.googleapis.com/benevolentmachines/e990_extract.csv'
            Data._e990 = pd.read_csv(u, sep='|', dtype=Data._e990_dt)
            _ = """
            u = 'https://drive.google.com/uc?export=download&id=' # download prefix
            e990_0 = pd.read_csv(u+'1fSjCPvNr7sr1e0ojjElHnWqyIIg2Xn_a', sep='|', dtype=Data._e990_dt)
            e990_1 = pd.read_csv(u+'13G7ThJQ__et7m9J7YvG0CKv3fQDIBIue', sep='|', dtype=Data._e990_dt)
            e990_2 = pd.read_csv(u+'13dk7CQQmVfUbnUhv9epRnm7E1hwVcDA7', sep='|', dtype=Data._e990_dt)
            Data._e990 = pd.concat([e990_0,e990_1,e990_2])
            """
        return Data._e990
    def get_ntee():
        import pandas as pd
        if Data._ntee is None:
            u = 'https://raw.githubusercontent.com/dhenderson/ntee/master/ntee.json'
            Data._ntee = pd.read_json(u, orient='index', dtype=Data._ntee_dt)
        return Data._ntee

    def get_gcst():
        import pandas as pd
        import numpy as np
        # https://taxonomy.candid.org/content/download/439387/9740582/file/Full_GCS_Taxo_Download.xlsx
        u = 'https://storage.googleapis.com/benevolent-machines/Full_GCS_Taxo_Download.xlsx'
        if Data._gcst is None:
            df = pd.read_excel(u, header=None)
            df.fillna('|', inplace=True)
            df['code'] = np.where(df[0] != '|', df[0], np.where(df[1] != '|', df[1], df[2]))
            df['desc'] = np.where(df[0] != '|', df[1], np.where(df[1] != '|', df[2], df[3]))
            df['desc'] = df['desc'].str.strip()
            Data._gcst = df[['code','desc']]
        return Data._gcst

    def _get_activities():
        import pandas as pd
        u = 'https://storage.googleapis.com/benevolent-machines/e990_activities.csv'
        if Data._activities is None:
            df = pd.read_csv(u, dtype={'ein': 'str'})
            Data._activities = df
        return Data._activities


In [ ]:
def heading():
    from IPython.core.display import HTML
    display(HTML("""
        <h1>title</h1>
        <h2 style="font-weight: normal; ">subtitle</h2>
    """))
#heading()

In [ ]:
def _study():
    eomf = Data.get_eomf()
    display(eomf[eomf['EIN'].isin(['274417979','200549531','581466516'])][[
        'EIN', 'NAME', 'CITY', 'STATE', 'RULING', 'TAX_PERIOD',
        'ASSET_AMT', 'INCOME_AMT', 'REVENUE_AMT', 'NTEE_CD']])

In [ ]:
def food_banks():
    eomf = Data.get_eomf()
    crit = eomf['NAME'].str.contains('FOOD BANK', na=False)
    cols = ['EIN','NAME','STREET','CITY','STATE','ZIP','ASSET_AMT','INCOME_AMT','REVENUE_AMT']
    display(eomf[crit][cols]) #.head(5)
#food_banks()

In [ ]:
def monthly_rulings():

    from datetime import datetime, timedelta
    from plotly.subplots import make_subplots
    import pandas as pd
    import plotly.express as px
    import plotly.graph_objs as go

    eomf = Data.get_eomf()
    eomf['RULING'].replace(['000000'],'190101', inplace=True)
    eomf['RULING'].replace(['190900'],'190901', inplace=True)
    eomf['RULING_DATE'] = pd.to_datetime(eomf['RULING'], format='%Y%m')

    ruling_counts = eomf[eomf['RULING'] > '200300']['RULING_DATE'].value_counts().sort_index()
    bar = go.Bar(x=ruling_counts.index, y=ruling_counts.values)
    fig = go.Figure()
    fig.add_trace(bar)
    max_date = pd.to_datetime('now', utc=True)
    fig.update_layout(
        xaxis=dict(
            title="Ruling",
            type="date",
            tick0=eomf.index[0], #dtick='M1',
            range= [max_date - timedelta(days=20*365), max_date],
            rangeselector=dict(
                buttons=list([
                    dict(label="20y", count=20, step="year", stepmode="backward"),
                    dict(label="10y", count=10, step="year", stepmode="backward"),
                    dict(label="5y", count=5, step="year", stepmode="backward"),
                    dict(label="2y", count=2, step="year", stepmode="backward"),
                    dict(label="1y", count=365+7, step="day", stepmode="backward")
                ]),
                x = 0,
                xanchor = 'left'
            ),
            rangeslider=dict(
                visible=True #,  range=[min_date, max_date]
            )
        ),
        updatemenus=[
            dict(
                type="buttons",
                showactive=False,
                buttons=[ ]
            )
        ],
        yaxis=dict(title='Count'),
        title='Count of Tax Exempt Rulings by Month',
        height=400
    )
    fig.update_xaxes(tickangle=45)
    fig.show()

In [ ]:
#test
monthly_rulings()

In [ ]:
def choropleth_by_state():
    import plotly.express as px
    import pandas as pd
    df = Data.get_eomf()
    sf = df[['STATE','REVENUE_AMT']].groupby('STATE').sum().reset_index()
    # Create the choropleth map
    fig = px.choropleth(sf,
        locations="STATE",
        color="REVENUE_AMT",
        locationmode="USA-states",
        scope="usa",
        color_continuous_scale=px.colors.sequential.Plasma,
        title="US Nonprofit Last Revenue by State")
    fig.show()
#choropleth_by_state()

In [ ]:
def exempt_org_pivot():
    from IPython.core.display import HTML
    import pandas as pd
    display(HTML("""<a name="eomf_pivot"></a>
    <h2>Exempt Organization Pivot</h2>
    """))
    TEMPLATE = u"""
<!DOCTYPE html>
<html>
    <head>
        <meta charset="UTF-8">
        <title>test</title>
        <link rel="stylesheet" type="text/css" href="https://cdnjs.cloudflare.com/ajax/libs/c3/0.4.11/c3.min.css">
        <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min.js"></script>
        <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/c3/0.4.11/c3.min.js"></script>
        <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/jquery/1.11.2/jquery.min.js"></script>
        <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/jqueryui/1.11.4/jquery-ui.min.js"></script>
        <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/jquery-csv/0.71/jquery.csv-0.71.min.js"></script>
        <link rel="stylesheet" type="text/css" href="https://cdnjs.cloudflare.com/ajax/libs/pivottable/2.19.0/pivot.min.css">
        <link rel="stylesheet" type="text/css" href="https://webpivot.nrecosite.com/Scripts/pivottable/nrecopivottableext.css">
        <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/pivottable/2.19.0/pivot.min.js"></script>
        <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/pivottable/2.19.0/d3_renderers.min.js"></script>
        <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/pivottable/2.19.0/c3_renderers.min.js"></script>
        <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/pivottable/2.19.0/export_renderers.min.js"></script>
        <script type="text/javascript" src="https://webpivot.nrecosite.com/Scripts/pivottable/nrecopivottableext.js"></script>
        <script type="text/javascript" src="https://cdn.plot.ly/plotly-basic-latest.min.js"></script>
        <style>
            body {font-family: Verdana;}
            .node {
                border: solid 1px white;
                font: 10px sans-serif;
                line-height: 12px;
                overflow: hidden;
                position: absolute;
                text-indent: 2px;
            }
            .c3-line, .c3-focused {stroke-width: 3px !important;}
            .c3-bar {stroke: white !important; stroke-width: 1;}
            .c3 text { font-size: 12px; color: grey;}
            .tick line {stroke: white;}
            .c3-axis path {stroke: grey;}
            .c3-circle { opacity: 1 !important; }
            .c3-xgrid-focus {visibility: hidden !important;}

            /* styles for responsive pivot UI table-layout: fixed; */
            table.pvtUi {
                width: 100%;
                table-layout: fixed;
            }
            table.pvtUi > tbody > tr:first-child > td:first-child {
                width: 200px;
            }
            table.pvtUi > tbody > tr:first-child > td:nth-child(2) {
                width: 200px;
            }
            .pvtTableRendererHolder {
                max-height: 540px;
            }
            .pvtRendererArea > div {
                overflow: auto;
            }
        </style>
    </head>
    <body style="padding: 0; margin: 0;">
        <div style="padding: 5px;">
            <span style='margin-left: 5px; font-size: 20px;'>IRS Exempt Master Stats</span>
            <span style='font-size: 14px;'> as of 2023-02-10</span>
        </div>
        <script type="text/javascript">
            $(function(){
                var income_avg = function() {
                  return function() {
                    return {
                      sumINCOME_TOT: 0.0,
                      sumCOUNT: 0.0,
                      push: function(record) {
                        if (!isNaN(parseFloat(record['INCOME_TOT']))) {
                          this.sumINCOME_TOT += parseFloat(record['INCOME_TOT']);
                        }
                        if (!isNaN(parseFloat(record['COUNT']))) {
                          return this.sumCOUNT += parseFloat(record['COUNT']);
                        }
                      },
                      value: function() {
                        if (this.sumCOUNT == 0) {
                          return 0.0;
                        }
                        return ( this.sumINCOME_TOT / this.sumCOUNT );
                      },
                      format: function(x) { return (x).toFixed(0).toString(); },
                      numInputs: 0
                    };
                  };
                };
                // deriver
                var ntee1 = {
                    "A": "Arts, Culture & Humanities",
                    "B": "Education",
                    "C": "Environment",
                    "D": "Animal-Related",
                    "E": "Health Care",
                    "F": "Mental Health & Crisis Intervention",
                    "G": "Voluntary Health Associations & Medical Disciplines",
                    "H": "Medical Research",
                    "I": "Crime & Legal-Related",
                    "J": "Employment",
                    "K": "Food, Agriculture & Nutrition",
                    "L": "Housing & Shelter",
                    "M": "Public Safety, Disaster Preparedness & Relief",
                    "N": "Recreation & Sports",
                    "O": "Youth Development",
                    "P": "Human Services",
                    "Q": "International, Foreign Affairs & National Security",
                    "R": "Civil Rights, Social Action & Advocacy",
                    "S": "Community Improvement & Capacity Building",
                    "T": "Philanthropy, Voluntarism & Grantmaking Foundations",
                    "U": "Science & Technology",
                    "V": "Social Science",
                    "W": "Public & Societal Benefit",
                    "X": "Religion-Related",
                    "Y": "Mutual & Membership Benefit",
                    "Z": "Unknown"
                }
                var filing_req = {
                    "00": "00: Not req. (other)",
                    "01": "01: Required 990/990EZ",
                    "02": "02: Required 990-N ",
                    "03": "03: Group return",
                    "04": "04: Required 990-BL ",
                    "06": "06: Not req. (church)",
                    "07": "07: Government 501(c)1",
                    "13": "13: Not req. (religious)",
                    "14": "14: Not req. (political)"
                }
                var range = {
                    "0": "0 $0",
                    "1": "1 $1 <$10K",
                    "2": "2 $10K <$25K",
                    "3": "3 $25K <$100K",
                    "4": "4 $100K <$500K",
                    "5": "5 $500K <$1M",
                    "6": "6 $1M <$5M",
                    "7": "7 $5M <$10M",
                    "8": "8 $10M <$50M",
                    "9": "9 $50M and over"
                }
                var NTEE1 = function(record) {
                    var s = ntee1[record["NTEE_CD_1"]];
                    return s;
                }
                var FILING_REQ = function(record) {
                    var s = filing_req[record["FILING_REQ_CD"]];
                    return s;
                }
                var ASSET = function(record) {
                    var s = range[record["ASSET_CD"]];
                    return s;
                }
                var INCOME = function(record) {
                    var s = range[record["INCOME_CD"]];
                    return s;
                }

                var nrecoPivotExt = new NRecoPivotTableExtensions({
                    fixedHeaders : true,
	                drillDownHandler: function (dataFilter) {
		                console.log(dataFilter);
                        alert(dataFilter);
	                },
                    wrapWith: '<div class="pvtTableRendererHolder"></div>',

                });

                var stdRendererNames = ["Table","Table Barchart","Heatmap","Row Heatmap","Col Heatmap"];
                var wrappedRenderers = $.extend( {},
                    $.pivotUtilities.renderers,
                    $.pivotUtilities.plotly_renderers,
                    $.pivotUtilities.c3_renderers
                );
                //{"GiftAmountGrowth": GiftAmountGrowthRate}, digitsAfterDecimal: 0

                var tpl = $.pivotUtilities.aggregatorTemplates;
                var numberFormat = $.pivotUtilities.numberFormat;
                var intFormat = numberFormat({digitsAfterDecimal: 0});
                var curFormat = numberFormat({prefix: '$', digitsAfterDecimal: 0});
                var aggregators = $.extend( {},
                    {"ORG_COUNT": function() { return tpl.sum(intFormat)(["COUNT"])}},
                    {"ASSET_TOT": function() { return tpl.sum(curFormat)(["ASSET_TOT"])}},
                    {"ASSET_AVG": function() { return tpl.sumOverSum(curFormat)(["ASSET_TOT", "COUNT"]); }},
                    {"INCOME_TOT": function() { return tpl.sum(curFormat)(["INCOME_TOT"])}},
                    {"INCOME_AVG": function() { return tpl.sumOverSum(curFormat)(["INCOME_TOT", "COUNT"]); }}
                    //$.pivotUtilities.aggregators
                    );
                $.each(stdRendererNames, function() {
                    var rName = this;
                    wrappedRenderers[rName] =
                        nrecoPivotExt.wrapPivotExportRenderer(
                            nrecoPivotExt.wrapTableRenderer(wrappedRenderers[rName]));
                });
                output_data = $.csv.toArrays($("#output").text());
                var pui = $('#output').pivotUI(output_data, {
                    aggregators: aggregators,
                    aggregatorName: "ORG_COUNT",
                    cols: ['ASSET'],
                    derivedAttributes: {
                        "NTEE": NTEE1,
                        "TAX_FILING_REQ": FILING_REQ,
                        "RANGE_ASSET": ASSET,
                        "RANGE_INCOME": INCOME
                    },
                    hiddenAttributes: ['','rownum','INCOME_CD','ASSET_CD','FILING_REQ_CD',
                                       'PF_FILING_REQ_CD', 'NTEE_CD_1',
                                       'SUBSECTION','AFFILIATION','FOUNDATION','STATUS',
                                       'DEDUCTIBILITY', 'ORGANIZATION'],
                    hiddenFromDragDrop: ['ASSET_TOT','INCOME_TOT','COUNT'],
                    hiddenFromAggregators: ['ACCT_PD','TAX_FILING_REQ','NTEE','RANGE_ASSET',
                                             'RANGE_INCOME','TAX_PERIOD_YEAR','RULING_DECADE'],
                    autoSortUnusedAttrs: true,
                    unusedAttrsVertical: true,
					          rendererName: "Table",
                    rendererOptions: {
                        table: {
                            clickCallback: function(e, value, filters, pivotData){
                                var names = [];
                                pivotData.forEachMatchingRecord(filters,
                                    function(record){ names.push(record.Name); });
                                alert(names.join("n"));
                            }
                        },
                        heatmap: {
                            colorScaleGenerator: function(values) {
                                // Plotly happens to come with d3 on board
                                return Plotly.d3.scale.linear()
                                    .domain([0, 1_500_000])
                                    .range(["#FFF", "#0F0"])
                            }
                        }
                    },
                    renderers: wrappedRenderers,
                    rows: ['INCOME'],
                    vals: ['COUNT'],
                    onRefresh: function (config) {
                        nrecoPivotExt.initFixedHeaders($('#output table.pvtTable'));

                        if (config.aggregatorName == 'Count') {
                            config.rendererOptions.heatmap.colorScaleGenerator = function(values) {
                                return Plotly.d3.scale.linear()
                                        .domain([0, 1_000])
                                        .range(["#FFF", "#0F0"])
                            }
                        }
                        if (config.aggregatorName == 'Integer Sum') {
                            config.rendererOptions.heatmap.colorScaleGenerator = function(values) {
                                return Plotly.d3.scale.linear()
                                        .domain([0, 1_000])
                                        .range(["#FFF", "#0F0"])
                            }
                        }
                    }
                }).show();
                nrecoPivotExt.initFixedHeaders($('#output table.pvtTable'));
            });
        </script>

        <div id="output" style="display: none;">__CSV__</div>
        </body>
    </html>
    """
    import json, io
    eomf = Data.get_eomf()
    eomf['NTEE_CD'].fillna('?', inplace = True)
    eomf['TAX_PERIOD'].fillna('????', inplace = True)
    eomf['RULING_YEAR'] = eomf['RULING'].str[0:4]
    eomf['RULING_DECADE'] = eomf['RULING'].str[0:3] + '0s'
    eomf['TAX_PERIOD_YEAR'] = eomf['TAX_PERIOD'].str[0:4]
    eomf['NTEE_CD_1'] = eomf['NTEE_CD'].str[0:1]

    agg_ASSET_TOT = pd.NamedAgg(column='ASSET_AMT', aggfunc='sum')
    agg_INCOME_TOT = pd.NamedAgg(column='INCOME_AMT', aggfunc='sum')
    agg_COUNT = pd.NamedAgg(column='EIN', aggfunc='count')
    df = eomf.groupby([
       'SUBSECTION', 'AFFILIATION',
       'DEDUCTIBILITY', 'FOUNDATION', 'ORGANIZATION', 'STATUS',
       'TAX_PERIOD_YEAR', 'ASSET_CD', 'INCOME_CD', 'FILING_REQ_CD',
       'PF_FILING_REQ_CD', 'ACCT_PD',
       'NTEE_CD_1','RULING_DECADE']).agg(
           ASSET_TOT=agg_ASSET_TOT,
           INCOME_TOT=agg_INCOME_TOT,
           COUNT=agg_COUNT).reset_index()

    csv = df.to_csv(encoding='utf8')
    # df.to_csv('weekly_gifts.csv')
    if hasattr(csv, 'decode'):
        csv = csv.decode('utf8')
    srcdoc = TEMPLATE.replace('__CSV__',csv)
    with open("exempt_org_pivot.html", "w") as text_file:
        text_file.write(srcdoc)
    srcdoc = srcdoc.replace('"','&quot;')  # encode quotes
    display(HTML(('''
      <iframe
          width="100%"
          height="500"
          srcdoc="''' + srcdoc + '''"
          frameborder="0"
          allowfullscreen
      ></iframe>
    ''')))
    """    """


In [ ]:
#test
exempt_org_pivot()